In [169]:
from gen_ai_hub.proxy import get_proxy_client
proxy_client = get_proxy_client(proxy_version="gen-ai-hub")
proxy_client.get_deployments()

[Deployment(url='https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc5b752a11e5fa73', config_id='1066614d-388f-4f86-8dcb-e8e37ed1fbe1', config_name='gpt-4o', deployment_id='dc5b752a11e5fa73', model_name='gpt-4o', created_at=datetime.datetime(2024, 7, 15, 8, 6, 34, tzinfo=datetime.timezone.utc), additonal_parameters={}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d7cb2d1ac98ffd3d', config_id='d010c3d2-61f8-4cce-922f-747747986e0e', config_name='Gemini', deployment_id='d7cb2d1ac98ffd3d', model_name='gemini-1.0-pro', created_at=datetime.datetime(2024, 6, 28, 14, 24, 10, tzinfo=datetime.timezone.utc), additonal_parameters={}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc34e5da947ad1d4', config_id='d010c3d2-61f8-4cce-922f-747747986e0e', config_name='Gemini', dep

In [170]:
from gen_ai_hub.proxy.langchain.init_models import init_llm

model = init_llm(model_name = "gpt-4o")

model.invoke("Are you there?")

APIConnectionError: Connection error.

In [ ]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

In [ ]:
class State(TypedDict):
    messages : Annotated[Sequence[BaseMessage], add_messages]
    langauge : str

graph_builder = StateGraph(state_schema=State)

In [ ]:
from langchain_core.messages import trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

messages = [
    ("system", "You are a helpful Ai who answers best to your ability in mentioned language {langauge}"),
    MessagesPlaceholder(variable_name="messages")
]

template = ChatPromptTemplate(
    messages=messages,
    input_variables = ["langauge"]
)

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
def chatbot(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    print(trim_messages)
    prompt = template.invoke(
        {"messages": trimmed_messages, "langauge": state["langauge"]}
    )
    print(prompt)
    response = model.invoke(prompt)
    return {"messages" : response}


In [ ]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import HumanMessage
config = {"configurable" : {"thread_id" : "rakesh1"}}
langauge = input("Enter the reponse langauge\n")
print(langauge)
while True:
    query = input("User: \n")
    print("Human: ", query)
    if query.lower() == "exit":
        print("Good Bye!!!")
        break
    input_messages = [HumanMessage(content=query)]
    print("AI: ",graph.invoke({"messages":input_messages, "langauge": langauge}, config)["messages"][-1].content)
    print("--" * 200)


english
Human:  what is ML


TypeError: 'ChatPromptValue' object is not subscriptable